In [1]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from IPython.display import Markdown, display, HTML

In [2]:
PROMPT_ROOT_DIR = "prompts"
TOPIC_DIR = "cons"
TOPIC = "Legal Services Department Questions"

In [3]:
from pathlib import Path

DEFAULT_PROMPT_ROOT_DIR = "prompts"
DEFAULT_PROMPT_FILE = "prompt_concierge.txt"
DEFAULT_TOPIC_DIR = "cons"
DEFAULT_TOPIC = "Legal Services Department Inquiries"


def build_omni_prompt(
        input_value: str,
        topic: str = DEFAULT_TOPIC,
        topic_dir: str = DEFAULT_TOPIC_DIR,
    ) -> str:
        """
        Constructs a complete prompt.

        Args:
            input_value (str): The user's input question or query.
            topic str: The specific topic under which the prompt is categorized.
            topic_dir str: The directory associated with the topic.

        Returns:
            str: A fully constructed prompt ready for use in the autocomplete system.
        """
        try:
            prompt_path = Path(DEFAULT_PROMPT_ROOT_DIR) / DEFAULT_PROMPT_FILE
            previous_completions_path = (
                Path(DEFAULT_PROMPT_ROOT_DIR)
                / "knowledge_bases"
                / topic_dir
                / "previous_completions.json"
            )
            domain_knowledge_path = (
                Path(DEFAULT_PROMPT_ROOT_DIR)
                / "knowledge_bases"
                / topic_dir
                / "domain_knowledge.txt"
            )

            with prompt_path.open("r") as file:
                prompt = file.read()

            with previous_completions_path.open("r") as file:
                previous_completions = file.read()

            with domain_knowledge_path.open("r") as file:
                domain_knowledge = file.read()

            prompt = prompt.replace("{{topic}}", topic)
            prompt = prompt.replace("{{previous_completions}}", previous_completions)
            prompt = prompt.replace("{{domain_knowledge}}", domain_knowledge)
            prompt = prompt.replace("{{input_value}}", input_value)

            return prompt

        except FileNotFoundError as e:
            raise FileNotFoundError(f"Required file not found: {e.filename}")
        except Exception as e:
            raise RuntimeError(f"An error occurred while building the prompt: {e}")

In [8]:
from textwrap import fill
from typing import List, Optional
from typing_extensions import Annotated
from annotated_types import Gt, Lt
from pydantic import BaseModel, Field, model_validator
from openai.types.completion_usage import CompletionUsage


class Prediction(BaseModel):
    """Predicted optimal department for a new user query."""
    
    chain_of_thought: str = Field(
        description="Reasoning behind the prediction based on PREVIOUS_PREDICTIONS and DOMAIN_KNOWLEDGE.",
        exclude=True,
    )
    predicted_department: str = Field(
        ...,
        description="The predicted department.",
    )
    confidence: Annotated[int, Gt(0), Lt(11)] = Field(
        ...,
        description="An integer score from 1-10 indicating prediction confidence.",
    )
    
    def __str__(self):
        wrapped_thought = fill(self.chain_of_thought, width=100)
        thought = f"Thought: {wrapped_thought}\n\n"
        thought += f"Predicted Department: {self.predicted_department}\n\n"
        thought += f"Score: {self.confidence}\n"
        return thought
        
        
    
class MultiPredict(BaseModel):
    """
    Class containing multiple (THREE) predictions.

    Args:
        predictions (List[Prediction]): The list of predicted departments.
    """

    predictions: List[Prediction] = Field(
        ...,
        description="List of predictions.",
    )
    
    @model_validator(mode='after')
    def sort_predictions_by_confidence(cls, values):
        values.predictions = sorted(values.predictions, key=lambda p: p.confidence, reverse=True)
        return values       
    
    @property
    def print_preds(self):
        output_string = ""
        for pred in self.predictions:
            output_string += str(pred)
            output_string += "\n------------------------------------------------\n\n"
        return output_string


class PredictionRequest(BaseModel):
    """Request model for making predictions."""
    
    user_query: str
    ground_truth: Optional[str] = None
    model_output: Optional[MultiPredict] = None
    token_usage: Optional[CompletionUsage] = None
    model_name: Optional[str] = None
    run_time: Optional[float] = None
    
    @property
    def cost(self):
        return self.token_usage.prompt_tokens * 5 / 1000000 + self.token_usage.completion_tokens * 15 / 1000000
    
    @property
    def prediction_rank(self) -> int:
        """
        Returns the rank of the ground truth department in the predictions list.

        Returns:
            int: The rank of the ground truth department, or 0 if not found.
        """
        if not self.ground_truth or not self.model_output:
            return 0
        
        for rank, prediction in enumerate(self.model_output.predictions, start=1):
            if prediction.predicted_department == self.ground_truth:
                return rank
        
        return 0
    
    @property
    def cost(self) -> float:
        """Calculates the cost based on token usage.

        Returns:
            float: The calculated cost.
        """
        return self.token_usage.prompt_tokens * 5 / 1000000 + self.token_usage.completion_tokens * 15 / 1000000
    
    @property
    def prediction_rank(self) -> int:
        """
        Returns the rank of the ground truth department in the predictions list.

        Returns:
            int: The rank of the ground truth department, or 0 if not found.
        """
        if not self.ground_truth or not self.model_output:
            return 0
        
        for rank, prediction in enumerate(self.model_output.predictions, start=1):
            if prediction.predicted_department == self.ground_truth:
                return rank
        
        return 0
    
    @property
    def correct_top_one(self) -> bool:
        """Checks if the ground truth department is the top prediction.

        Returns:
            bool: True if the ground truth department is the top prediction, else False.
        """
        return self.prediction_rank == 1
    
    @property
    def correct_top_two(self) -> bool:
        """Checks if the ground truth department is within the top two predictions.

        Returns:
            bool: True if the ground truth department is within the top two predictions, else False.
        """
        return self.prediction_rank in [1, 2]

In [9]:
import instructor
import openai
import time

def get_predictions(input_data: PredictionRequest, topic: str = DEFAULT_TOPIC, topic_dir: str = DEFAULT_TOPIC_DIR) -> MultiPredict:
    start_time = time.time()
    input_value = input_data.user_query
    prompt = build_omni_prompt(
        input_value=input_value, topic=topic, topic_dir=topic_dir
    )
    client = instructor.from_openai(openai.OpenAI())
    
    response, completion = client.chat.completions.create_with_completion(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            model="gpt-4o",
            response_model=MultiPredict,
        )
    # response.sort_predictions_by_confidence()
    return PredictionRequest(
        user_query=input_data.user_query,
        ground_truth=input_data.ground_truth or None,
        model_output=response,
        token_usage=completion.usage,
        model_name=completion.model,
        run_time=time.time() - start_time
    )

In [10]:
query = "I have a complex claim that requires representation in court."

request = PredictionRequest(
    user_query=query,
    ground_truth="Field Legal",
)

response = get_predictions(request)

In [15]:
response.cost

0.009264999999999999

In [14]:
response.model_dump()

{'user_query': 'I have a complex claim that requires representation in court.',
 'ground_truth': 'Field Legal',
 'model_output': {'predictions': [{'predicted_department': 'Field Legal',
    'confidence': 10},
   {'predicted_department': 'Coverage Team', 'confidence': 8},
   {'predicted_department': 'Professional Liability Group', 'confidence': 7}]},
 'token_usage': {'completion_tokens': 203,
  'prompt_tokens': 1244,
  'total_tokens': 1447},
 'model_name': 'gpt-4o-2024-05-13',
 'run_time': 3.1944096088409424}

In [16]:
print(response.model_output.print_preds)

Thought: The user's request for representation in court suggests litigation. According to the
DOMAIN_KNOWLEDGE, the 'Field Legal' department handles all litigation matters. This includes cases
referred from the Coverage Team and other departments, indicating their specialization in defending
the company in court.

Predicted Department: Field Legal

Score: 10

------------------------------------------------

Thought: Since the user mentioned a 'complex claim,' it could involve intricate policy definitions and case
law research before heading to litigation. The 'Coverage Team' provides specialized attorneys for
such advice on complex claims, making it a relevant department to consider.

Predicted Department: Coverage Team

Score: 8

------------------------------------------------

Thought: If the complex claim involves professional liability, such as issues against healthcare, legal, or
financial professionals, the 'Professional Liability Group' would be an appropriate choice. They
man

In [11]:
response

PredictionRequest(user_query='I have a direct report who advised me she is being harassed by a coworker.', ground_truth='Compliance Team', model_output=MultiPredict(predictions=[Prediction(chain_of_thought='The new INPUT_VALUE relates to an employee reporting harassment by a coworker and could involve internal company policies and potential regulatory concerns. Based on the DOMAIN_KNOWLEDGE regarding which department handles internal disputes and regulatory compliance, and given the lack of precise matches in the PREVIOUS_PREDICTIONS, the Compliance Team would be a primary point of contact here.', predicted_department='Compliance Team', confidence=9), Prediction(chain_of_thought="Considering the user’s report of workplace harassment, it can also touch upon legal aspects of employment and the potential need for internal investigations. Thus, the Employment Disputes department, which handles issues concerning employees pursuing actions against the company, is relevant here, even though i

In [3]:
prompt = open(f"{PROMPT_ROOT_DIR}/prompt_concierge.txt", "r").read()
print(prompt)

# You're a world class legal concierge AI system that predicts optimal queries to COMPLETE the goal of routing them to the correct department.
You take a TOPIC, PREVIOUS_COMPLETIONS from past interactions, and DOMAIN_KNOWLEDGE to generate the most likely department and best phrased query based on user INPUT_VALUE.

You closely follow GENERATION_RULES to provide the best possible outcomes.

## GENERATION_RULES
- If the users INPUT_VALUE exists within their PREVIOUS_COMPLETIONS, prefer that completion. Always prefer completions with more hits.
- If the users INPUT_VALUE does NOT exist in PREVIOUS_COMPLETIONS, predict completions from DOMAIN_KNOWLEDGE.
- Your completions should should always be a valid sentence to be used for future examples.
- Be sure to follow the correct format.

## TOPIC
{{topic}}

## PREVIOUS_COMPLETIONS
{{previous_completions}}

## DOMAIN_KNOWLEDGE
{{domain_knowledge}}

## Complete the following INPUT_VALUE
{{input_value}}



In [4]:
from pathlib import Path
from typing import Optional

def build_omni_complete_prompt(input_value: str, topic: str = "Legal Services Department Intake", topic_dir: str = "cons") -> str:
    """
    Constructs a complete prompt for the Omni autocomplete system by incorporating various components.

    Args:
        input_value (str): The user's input question or query.
        topic str: The specific topic under which the prompt is categorized. Defaults to "Legal Services Department Intake".
        topic_dir str: The directory associated with the topic. Defaults to "cons".

    Returns:
        str: A fully constructed prompt ready for use in the autocomplete system.
    """
    try:
        prompt_path = Path(PROMPT_ROOT_DIR) / "prompt_concierge.txt"
        previous_completions_path = Path(PROMPT_ROOT_DIR) / "knowledge_bases" / topic_dir / "previous_completions.json"
        domain_knowledge_path = Path(PROMPT_ROOT_DIR) / "knowledge_bases" / topic_dir / "domain_knowledge.txt"

        with prompt_path.open("r") as file:
            prompt = file.read()

        with previous_completions_path.open("r") as file:
            previous_completions = file.read()

        with domain_knowledge_path.open("r") as file:
            domain_knowledge = file.read()

        prompt = prompt.replace("{{topic}}", topic)
        prompt = prompt.replace("{{previous_completions}}", previous_completions)
        prompt = prompt.replace("{{domain_knowledge}}", domain_knowledge)
        prompt = prompt.replace("{{input_value}}", input_value)

        return prompt

    except FileNotFoundError as e:
        raise FileNotFoundError(f"Required file not found: {e.filename}")
    except Exception as e:
        raise RuntimeError(f"An error occurred while building the prompt: {e}")

In [5]:
prompt_example = build_omni_complete_prompt("who can help me with a question about pollution damage?")
print(prompt_example)

# You're a world class legal concierge AI system that predicts optimal queries to COMPLETE the goal of routing them to the correct department.
You take a TOPIC, PREVIOUS_COMPLETIONS from past interactions, and DOMAIN_KNOWLEDGE to generate the most likely department and best phrased query based on user INPUT_VALUE.

You closely follow GENERATION_RULES to provide the best possible outcomes.

## GENERATION_RULES
- If the users INPUT_VALUE exists within their PREVIOUS_COMPLETIONS, prefer that completion. Always prefer completions with more hits.
- If the users INPUT_VALUE does NOT exist in PREVIOUS_COMPLETIONS, predict completions from DOMAIN_KNOWLEDGE.
- Your completions should should always be a valid sentence to be used for future examples.
- Be sure to follow the correct format.

## TOPIC
Legal Services Department Intake

## PREVIOUS_COMPLETIONS
[
    {
        "input": "i need help with a wrongful termination case",
        "completions": [
            "Seeking guidance on managing a 

In [ ]:
from typing import Any, Dict, List
import tiktoken


def truncate_json_by_tokens(json_string: str, threshold: int, model: str = "gpt-3.5-turbo") -> List[Dict[str, Any]]:
    """
    Truncates a JSON string into a list of dictionaries based on a token threshold using a specified model.

    Args:
        json_string (str): The JSON string to be truncated.
        threshold (int): The maximum number of tokens allowed in the truncated output.
        model (str): The model used to count tokens, default is "gpt-3.5-turbo".

    Returns:
        List[Dict[str, Any]]: A list of dictionaries representing the truncated JSON data.
    """
    data = json.loads(json_string)
    truncated_data = []
    current_tokens = 0

    for item in data:
        item_string = json.dumps(item, indent=4)
        item_tokens = count_tokens(item_string, model)
        
        if current_tokens + item_tokens > threshold:
            break
        
        truncated_data.append(item)
        current_tokens += item_tokens

    return truncated_data

In [6]:
from typing import List, Type, TypeVar
from pydantic import BaseModel

T = TypeVar('T', bound=BaseModel)


def save_models_to_json(models: List[BaseModel], file_path: str) -> None:
    """
    Saves a list of Pydantic models to a JSON file.

    Args:
        models: A list of Pydantic model instances.
        file_path: The path to the JSON file where the data will be saved.

    Returns:
        None
    """
    try:
        with open(file_path, 'w') as f:
            # Convert list of models to list of dictionaries
            data = [model.model_dump() for model in models]
            json.dump(data, f, indent=4)
    except FileNotFoundError:
        raise FileNotFoundError(f"Required file not found: {file_path}")
    except Exception as e:
        raise RuntimeError(f"An error occurred while loading the previous completions: {e}")  
        

def load_models_from_json(model_class: Type[T], file_path: str) -> List[T]:
    """
    Loads JSON data from a file and converts it into a list of Pydantic models.

    Args:
        model_class: The Pydantic model class to which the JSON objects will be converted.
        file_path: The path to the JSON file from which the data will be loaded.

    Returns:
        A list of Pydantic model instances.
    """
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            models = [model_class.model_validate(item) for item in data]
        return models
    except FileNotFoundError:
        raise FileNotFoundError(f"Required file not found: {file_path}")
    except Exception as e:
        raise RuntimeError(f"An error occurred while loading the previous completions: {e}")  

In [37]:
def get_completions_path(prompt_root_dir: str, topic_dir: str) -> str:
    return f"{prompt_root_dir}/knowledge_bases/{topic_dir}/previous_completions.json"

In [39]:
prompt_path = get_completions_path(PROMPT_ROOT_DIR, "cons")

previous_comp_models = load_models_from_json(
    model_class=AutoCompletion,
    file_path=prompt_path,
)

In [54]:
from pydantic import BaseModel, Field
from typing import List

class AutoCompletionEntry(BaseModel):
    input: str
    completions: List[str]
    correct_department: str
    hits: int = 1
    
    def __str__(self):
        return self.model_dump_json(indent=4)
    
    @property
    def token_count(self):
        return count_tokens(str(self))
    
    
    

class AutoCompletions(BaseModel):
    """Auto-completions for a new user query."""
    
    input: str = Field(
        ...,
        description="The user provided INPUT_VALUE.",
    )
    completions: List[str] = Field(
        default_factory=list,
        description="A list of potential completions based on the GENERATION_RULES.",
    )
    correct_department: str = Field(
        ...,
        description="The predicted department based on ALL available information.",
    )

In [55]:
import json
from typing import List
from pydantic import ValidationError

def increment_or_create_previous_completions(input: str, completion: str, topic_dir: str) -> List[AutoCompletionEntry]:
    """
    Updates the list of previous completions by incrementing the hit count for existing entries
    or creating a new entry if no match is found. The list is then sorted by hits in descending order
    and saved back to the file.

    Args:
        input (str): The user input string to match or add.
        completion (str): The completion string associated with the input.
        topic_dir (str): The directory name under which the completions file is stored.

    Returns:
        List[AutoCompletionEntry]: A list of AutoCompletionEntry objects sorted by hits in descending order.
    """
    previous_completions_file = (
        f"{PROMPT_ROOT_DIR}/knowledge_bases/{topic_dir}/previous_completions.json"
    )

    # Try to read the previous completions file
    try:
        with open(previous_completions_file, "r") as file:
            data = json.load(file)
            previous_completions = [AutoCompletionEntry(**item) for item in data]
    except (FileNotFoundError, json.JSONDecodeError, ValidationError):
        previous_completions = []

    # Check for a matching input and completion
    matching_case = None
    for item in previous_completions:
        if item.input.lower() == input.lower():
            for existing_completion in item.completions:
                if completion.lower() in existing_completion.lower():
                    matching_case = item
                    break

    # Update hits if a match is found, otherwise create a new entry
    if matching_case:
        matching_case.hits += 1
    else:
        new_completion = AutoCompletionEntry(input=input, completions=[completion])
        previous_completions.append(new_completion)

    # Sort completions by hits in descending order
    completions_sorted_by_hits = sorted(
        previous_completions, key=lambda x: x.hits, reverse=True
    )

    # Write the updated completions back to the file
    with open(previous_completions_file, "w") as file:
        json.dump([item.model_dump() for item in completions_sorted_by_hits], file, indent=4)

    return completions_sorted_by_hits

In [57]:
import instructor
import openai

def get_autocompletions(input_data: str, topic: str, topic_dir: str) -> AutoCompletions:

    prompt = build_omni_complete_prompt(
        input_data, topic=topic, topic_dir=topic_dir
    )
    client = instructor.from_openai(openai.OpenAI())
    
    return client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            model="gpt-4o",
            response_model=AutoCompletions,
        )

In [59]:
input_data = 'I need assistance with a legal issue involving a NDA'

autocompletions = get_autocompletions(
    input_data=input_data, 
    topic="Legal Services Department Intake", 
    topic_dir="cons",
)

In [61]:
autocompletions.model_dump()

{'input': 'I need assistance with a legal issue involving a NDA',
 'completions': ['Need assistance with a legal issue involving a non-disclosure agreement (NDA).'],
 'correct_department': 'Contractual Disputes Team'}